In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# if using mac
df = pd.read_excel('/Users/zhanghantang/PycharmProjects/sucide_project/BIOM40forUSC.xlsx')
data = df[df['SI'].notnull()]
y = data['SI']
x = data.loc[:, 'GIMAP1Biom1552316_a_at':'CFIS']

In [71]:
# windows
df = pd.read_excel('E:\PHD\sucide_project\BIOM40forUSC.xlsx')
data = df[df['SI'].notnull()]
y = data['SI']
x = data.loc[:, 'GIMAP1Biom1552316_a_at':'CFIS']

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\PHD\\sucide_project\\BIOM40forUSC.xlsx'

In [4]:
# column 'CFI-S.PheneVisit' data type is string, i am not clear its internal meaning and how to convert to float data type
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
x = x.drop(labels='CFI-S.PheneVisit', axis=1)
# drop these column directly
x = data.loc[:, 'GIMAP1Biom1552316_a_at':'RAB3GAP2Biom240234_at']
new_y = []
for i in y:
    if i == 0 or i ==1:
        new_y.append(0)
    else:
        new_y.append(1)
y = np.array(new_y)
X = StandardScaler().fit_transform(x)
X_train, X_test, y_train, y_test = train_test_split(X, new_y, test_size=0.2, random_state=10)
print(len(X_train), len(X_test))

# todo unified randome state
unified_random_state = 32

603 151


In [ ]:
'''
-------------------------
the content below is about logistic regression content
-------------------------
'''

In [108]:
# tune parameter
from sklearn.model_selection import GridSearchCV
model = LR(solver='saga')
# 不同的solver可能只支持不同的有限的penalty
param_grid = {'penalty':['l1', 'l2'], 'C':[0.1, 1, 10, 100], 'max_iter':[3000, 3500]}
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1', return_train_score=True)
grid_search.fit(X, new_y)
print("Best penalty: ", grid_search.best_params_['penalty'])
print("Best C: ", grid_search.best_params_['C'])
print("Best score: ", grid_search.best_score_)

'''
Best penalty:  l1
Best C:  100
Best score:  0.19299478970021897

'''

Best penalty:  l1
Best C:  100
Best score:  0.19299478970021897


In [206]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression as LR

scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
lr_model = LR(solver='saga', penalty='l1', C=100, max_iter=3000)
scores = cross_validate(lr_model, X, y, cv=5, scoring=scoring)
print("Scores: accuracy is {}, mean accuary is {}".format(scores['test_accuracy'], scores['test_accuracy'].mean()))
print("Scores: precision is {}, mean precision is {}".format(scores['test_precision'], scores['test_precision'].mean()))
print("Scores: recall is {}, mean recall is {}".format(scores['test_recall'], scores['test_recall'].mean()))
print("Scores: f1 is {}, mean f1 is {}".format(scores['test_f1'], scores['test_f1'].mean()))
print("Scores: roc_auc is {}, mean roc_auc is {}".format(scores['test_roc_auc'], scores['test_roc_auc'].mean()))


'''
Scores: accuracy is [0.8013245  0.76821192 0.74834437 0.76821192 0.73333333], mean accuary is 0.7638852097130242
Scores: precision is [0.44444444 0.25       0.09090909 0.23076923 0.35      ], mean precision is 0.2732245532245532
Scores: recall is [0.13793103 0.10344828 0.03448276 0.10714286 0.5       ], mean recall is 0.17660098522167486
Scores: f1 is [0.21052632 0.14634146 0.05       0.14634146 0.41176471], mean f1 is 0.19299478970021897
Scores: roc_auc is [0.5607688  0.63793103 0.58790277 0.67799071 0.7192623 ], mean roc_auc is 0.6367711207799311
'''

Scores: accuracy is [0.8013245  0.76821192 0.74834437 0.76821192 0.73333333], mean accuary is 0.7638852097130242
Scores: precision is [0.44444444 0.25       0.09090909 0.23076923 0.35      ], mean precision is 0.2732245532245532
Scores: recall is [0.13793103 0.10344828 0.03448276 0.10714286 0.5       ], mean recall is 0.17660098522167486
Scores: f1 is [0.21052632 0.14634146 0.05       0.14634146 0.41176471], mean f1 is 0.19299478970021897
Scores: roc_auc is [0.5607688  0.63793103 0.58790277 0.67799071 0.7192623 ], mean roc_auc is 0.6367711207799311


'\nScore with L1 penalty: accuracy:0.7831325301204819 precision:0.4074074074074074 recall:0.22448979591836735 f1:0.2894736842105263 roc_auc:0.5722448979591838\n'

In [ ]:
'''
-------------------------
the content below is about SVM
-------------------------
'''

In [145]:
from sklearn.model_selection import GridSearchCV
model = SVC(kernel='rbf', max_iter=10000)
param_grid = {'C':[1, 10, 100], }
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1', return_train_score=True)
grid_search.fit(X, y)

# print("Best penalty: ", grid_search.best_params_['kernel'])
print("Best C: ", grid_search.best_params_['C'])
print("Best score: ", grid_search.best_score_)

'''
Best penalty:  rbf
Best C:  100

Scores:  [0.36065574 0.44067797 0.45901639 0.31111111 0.40740741]
Mean Score:  0.39577372315355086
Scores:  [0.74274194 0.71833931 0.72649573 0.70439633 0.69226137]
Mean Score:  0.7168469323092028

'''

Best C:  100
Best score:  0.26710847637761254


'\nBest penalty:  rbf\nBest C:  10\nScores:  [0.41666667 0.3902439  0.45454545 0.30769231 0.38596491]\nMean Score:  0.391022648724831\n'

In [ ]:
'''
-------------------------
the content below is about SVM
-------------------------
'''
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

model = SVC(C=100, kernel='rbf', max_iter=10000)
scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
scores = cross_validate(model, X, y, cv=5, scoring=scoring)
# y_pred = model.predict(X)
print("Scores: accuracy is {}, mean accuary is {}".format(scores['test_accuracy'], scores['test_accuracy'].mean()))
print("Scores: precision is {}, mean accuary is {}".format(scores['test_precision'], scores['test_precision'].mean()))
print("Scores: recall is {}, mean accuary is {}".format(scores['test_recall'], scores['test_recall'].mean()))
print("Scores: f1 is {}, mean accuary is {}".format(scores['test_f1'], scores['test_f1'].mean()))
print("Scores: roc_auc is {}, mean accuary is {}".format(scores['test_roc_auc'], scores['test_roc_auc'].mean()))

'''
Scores: accuracy is [0.7615894  0.74834437 0.74172185 0.7615894  0.58      ], mean accuary is 0.7186490066225166
Scores: precision is [0.29411765 0.15384615 0.33333333 0.33333333 0.26027397], mean accuary is 0.27498088803487675
Scores: recall is [0.17241379 0.06896552 0.34482759 0.28571429 0.67857143], mean accuary is 0.31009852216748773
Scores: f1 is [0.2173913  0.0952381  0.33898305 0.30769231 0.37623762], mean accuary is 0.26710847637761254
Scores: roc_auc is [0.56783493 0.60655738 0.65008479 0.71544715 0.69818501], mean accuary is 0.6476218543781174

'''

In [ ]:
'''
-------------------------
the content below is about random forest
-------------------------
'''

In [224]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

param_grid = {'n_estimators':[10, 100, 200], 'max_depth':[8, 11, 13], 'min_samples_split': [4, 6, 8, 10],
              'min_samples_leaf': [2, 4, 6]}

skf = StratifiedKFold(n_splits=5)
rf_model = RandomForestClassifier(random_state=unified_random_state)
grid_search = GridSearchCV(rf_model, param_grid, cv=skf, scoring='roc_auc')
grid_search.fit(X, y)

print(grid_search.best_params_)

{'max_depth': 11, 'min_samples_leaf': 2, 'min_samples_split': 6, 'n_estimators': 200}


In [226]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, make_scorer

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision':make_scorer(precision_score, zero_division=0),
    'recall' :make_scorer(recall_score),
    'f1':make_scorer(f1_score),
    'roc_auc':make_scorer(roc_auc_score)
}
skf = StratifiedKFold(n_splits=5)

rf_model = RandomForestClassifier(n_estimators=200, max_depth=11, min_samples_leaf=2, min_samples_split=6, random_state=unified_random_state)

# scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

scores = cross_validate(rf_model, X, y, cv=skf, scoring=scoring)
print("Scores: accuracy is {}, mean accuary is {}".format(scores['test_accuracy'], scores['test_accuracy'].mean()))
print("Scores: precision is {}, mean precision is {}".format(scores['test_precision'], scores['test_precision'].mean()))
print("Scores: recall is {}, mean recall is {}".format(scores['test_recall'], scores['test_recall'].mean()))
print("Scores: f1 is {}, mean f1 is {}".format(scores['test_f1'], scores['test_f1'].mean()))
print("Scores: roc_auc is {}, mean roc_auc is {}".format(scores['test_roc_auc'], scores['test_roc_auc'].mean()))

# print("Score with L1 penalty: accuracy:{} precision:{} recall:{} f1:{} roc_auc:{}".format(accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred), roc_auc_score(y_test, y_pred)))
'''
200， 11， 2， 6
Scores: accuracy is [0.80794702 0.81456954 0.82119205 0.82119205 0.76666667], mean accuary is 0.8063134657836644
Scores: precision is [0.         1.         1.         1.         0.39393939], mean precision is 0.6787878787878788
Scores: recall is [0.         0.03448276 0.06896552 0.03571429 0.46428571], mean recall is 0.12068965517241378
Scores: f1 is [0.         0.06666667 0.12903226 0.06896552 0.42622951], mean f1 is 0.13817879003385666
Scores: roc_auc is [0.5        0.51724138 0.53448276 0.51785714 0.65017564], mean roc_auc is 0.543951384963256

'''


Scores: accuracy is [0.8013245  0.81456954 0.82119205 0.82119205 0.76      ], mean accuary is 0.8036556291390727
Scores: precision is [0.         1.         1.         1.         0.38235294], mean precision is 0.6764705882352942
Scores: recall is [0.         0.03448276 0.06896552 0.03571429 0.46428571], mean recall is 0.12068965517241378
Scores: f1 is [0.         0.06666667 0.12903226 0.06896552 0.41935484], mean f1 is 0.1368038561364479
Scores: roc_auc is [0.49590164 0.51724138 0.53448276 0.51785714 0.64607728], mean roc_auc is 0.542312040700961


'\n200， 11， 2， 6\nScores: accuracy is [0.80794702 0.81456954 0.82119205 0.82119205 0.76666667], mean accuary is 0.8063134657836644\nScores: precision is [0.         1.         1.         1.         0.39393939], mean precision is 0.6787878787878788\nScores: recall is [0.         0.03448276 0.06896552 0.03571429 0.46428571], mean recall is 0.12068965517241378\nScores: f1 is [0.         0.06666667 0.12903226 0.06896552 0.42622951], mean f1 is 0.13817879003385666\nScores: roc_auc is [0.5        0.51724138 0.53448276 0.51785714 0.65017564], mean roc_auc is 0.543951384963256\n\n'

In [ ]:
'''
-------------------------
the content below is about xgboost
-------------------------
'''

In [227]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5)

clf = XGBClassifier(random_state=unified_random_state)
param_grid = {'n_estimators':[10, 100, 200], 'max_depth':[4, 6, 8], 'learning_rate': [0.3, 0.01]}
grid_search = GridSearchCV(clf, param_grid, cv=skf, scoring='roc_auc')
grid_search.fit(X, y)

print(grid_search.best_params_)


{'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 200}


In [262]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, make_scorer

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision':make_scorer(precision_score, zero_division=0),
    'recall' :make_scorer(recall_score),
    'f1':make_scorer(f1_score),
    'roc_auc':make_scorer(roc_auc_score)
}
clf = XGBClassifier(n_estimators=250, learning_rate=0.07, max_depth=4, subsample=0.9,
                    colsample_bytree=0.8, reg_alpha=0.5)
skf = StratifiedKFold(n_splits=5)
scores = cross_validate(clf, X, y, cv=skf, scoring=scoring)
print("Scores: accuracy is {}, mean accuary is {}".format(scores['test_accuracy'], scores['test_accuracy'].mean()))
print("Scores: precision is {}, mean precision is {}".format(scores['test_precision'], scores['test_precision'].mean()))
print("Scores: recall is {}, mean recall is {}".format(scores['test_recall'], scores['test_recall'].mean()))
print("Scores: f1 is {}, mean f1 is {}".format(scores['test_f1'], scores['test_f1'].mean()))
print("Scores: roc_auc is {}, mean roc_auc is {}".format(scores['test_roc_auc'], scores['test_roc_auc'].mean()))

'''
n_estimators=250, learning_rate=0.07, max_depth=4, subsample=0.9,
                    colsample_bytree=0.8, reg_alpha=0.5
Scores: accuracy is [0.82781457 0.79470199 0.81456954 0.8013245  0.75333333], mean accuary is 0.7983487858719647
Scores: precision is [0.8        0.33333333 0.54545455 0.4        0.39534884], mean precision is 0.49482734319943616
Scores: recall is [0.13793103 0.06896552 0.20689655 0.14285714 0.60714286], mean recall is 0.23275862068965517
Scores: f1 is [0.23529412 0.11428571 0.3        0.21052632 0.47887324], mean f1 is 0.2677958774317733
Scores: roc_auc is [0.56486716 0.51808932 0.58295647 0.54703833 0.69701405], mean roc_auc is 0.5819930648430282
'''

Scores: accuracy is [0.82119205 0.78145695 0.80794702 0.79470199 0.71333333], mean accuary is 0.7837262693156732
Scores: precision is [0.66666667 0.16666667 0.5        0.36363636 0.34042553], mean precision is 0.40747904577691807
Scores: recall is [0.13793103 0.03448276 0.13793103 0.14285714 0.57142857], mean recall is 0.20492610837438424
Scores: f1 is [0.22857143 0.05714286 0.21621622 0.20512821 0.42666667], mean f1 is 0.22674507474507472
Scores: roc_auc is [0.5607688  0.49674958 0.55257207 0.54297329 0.65866511], mean roc_auc is 0.5623457677684263


'\nn_estimators=250, learning_rate=0.01, max_depth=4, subsample=0.9,\n                    colsample_bytree=0.8,\nSScores: accuracy is [0.82781457 0.79470199 0.81456954 0.8013245  0.75333333], mean accuary is 0.7983487858719647\nScores: precision is [0.8        0.33333333 0.54545455 0.4        0.39534884], mean precision is 0.49482734319943616\nScores: recall is [0.13793103 0.06896552 0.20689655 0.14285714 0.60714286], mean recall is 0.23275862068965517\nScores: f1 is [0.23529412 0.11428571 0.3        0.21052632 0.47887324], mean f1 is 0.2677958774317733\nScores: roc_auc is [0.56486716 0.51808932 0.58295647 0.54703833 0.69701405], mean roc_auc is 0.5819930648430282\n'

In [ ]:
'''
-------------------------
the content below is about native bayes
-------------------------
'''

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)

nb = GaussianNB()
# gaussian nb does not have many tunable hyparameters
param_grid = {'n_estimators':[10, 100, 200], 'max_depth':[4, 6, 8], 'learning_rate': [0.3, 0.01]}
grid_search = GridSearchCV(clf, param_grid, cv=skf, scoring='roc_auc')
grid_search.fit(X, y)

print(grid_search.best_params_)

In [265]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import GaussianNB

skf = StratifiedKFold(n_splits=5)

nb = GaussianNB()
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision':make_scorer(precision_score, zero_division=0),
    'recall' :make_scorer(recall_score),
    'f1':make_scorer(f1_score),
    'roc_auc':make_scorer(roc_auc_score)
}

skf = StratifiedKFold(n_splits=5)
scores = cross_validate(nb, X, y, cv=skf, scoring=scoring)
print("Scores: accuracy is {}, mean accuary is {}".format(scores['test_accuracy'], scores['test_accuracy'].mean()))
print("Scores: precision is {}, mean precision is {}".format(scores['test_precision'], scores['test_precision'].mean()))
print("Scores: recall is {}, mean recall is {}".format(scores['test_recall'], scores['test_recall'].mean()))
print("Scores: f1 is {}, mean f1 is {}".format(scores['test_f1'], scores['test_f1'].mean()))
print("Scores: roc_auc is {}, mean roc_auc is {}".format(scores['test_roc_auc'], scores['test_roc_auc'].mean()))

'''
Scores: accuracy is [0.56291391 0.54966887 0.37086093 0.2384106  0.38      ], mean accuary is 0.42037086092715226
Scores: precision is [0.1509434  0.24675325 0.17       0.16793893 0.20183486], mean precision is 0.18749408733253856
Scores: recall is [0.27586207 0.65517241 0.5862069  0.78571429 0.78571429], mean recall is 0.6177339901477832
Scores: f1 is [0.19512195 0.35849057 0.26356589 0.27672956 0.32116788], mean f1 is 0.2830151703380446
Scores: roc_auc is [0.4535048  0.58988129 0.45293951 0.44976771 0.53629977], mean roc_auc is 0.4964786170917511

'''

Scores: accuracy is [0.56291391 0.54966887 0.37086093 0.2384106  0.38      ], mean accuary is 0.42037086092715226
Scores: precision is [0.1509434  0.24675325 0.17       0.16793893 0.20183486], mean precision is 0.18749408733253856
Scores: recall is [0.27586207 0.65517241 0.5862069  0.78571429 0.78571429], mean recall is 0.6177339901477832
Scores: f1 is [0.19512195 0.35849057 0.26356589 0.27672956 0.32116788], mean f1 is 0.2830151703380446
Scores: roc_auc is [0.4535048  0.58988129 0.45293951 0.44976771 0.53629977], mean roc_auc is 0.4964786170917511


In [ ]:
'''
-------------------------
the content below is about dnn
-------------------------
'''

In [274]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau


# Define evaluation function
def evaluate(model, criterion, dataloader, device):
    model.eval()
    running_loss = 0.0
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels.float().view(-1, 1))
            running_loss += loss.item() * inputs.size(0)
            predicted = (outputs > 0.5).float()
            y_true.extend(labels.tolist())
            y_pred.extend(predicted.tolist())

    epoch_loss = running_loss / len(dataloader.dataset)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=1.0)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred)

    return epoch_loss, accuracy, precision, recall, f1, roc_auc

# Training loop
def train(model, criterion, optimizer, scheduler, train_dataloader, val_dataloader, device, num_epochs=10):
    # best_val_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.float().view(-1, 1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_dataloader.dataset)
        val_loss, accuracy, precision, recall, f1, roc_auc = evaluate(model, criterion, val_dataloader, device)
        scheduler.step(val_loss)

        print(f'Epoch {epoch+1}/{num_epochs}, '
              f'Training Loss: {epoch_loss:.4f}, '
              f'Validation Loss: {val_loss:.4f}, '
              f'Accuracy: {accuracy:.4f}, '
              f'Precision: {precision:.4f}, '
              f'Recall: {recall:.4f}, '
              f'F1-score: {f1:.4f}, '
              f'ROC AUC: {roc_auc:.4f}')

        # if val_loss < best_val_loss:
        #     best_val_loss = val_loss
        #     torch.save(model.state_dict(), 'best_model.pth')

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

(754, 66)

In [ ]:
# Set the minimum number of positive samples per split will reduce the performance

X_dnn = torch.tensor(X).float()
y_dnn = torch.tensor(y).float()
# Split the data into training and validation sets
dataset = TensorDataset(X_dnn, y_dnn)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

In [ ]:
input_size = x.shape[1]

class BinaryClassifier(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.dropout1(x)
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.relu(self.fc5(x))
        x = self.fc6(x)
        x = self.sigmoid(x)
        return x

model = BinaryClassifier(input_size)
# commonly use in binary classification
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=400, verbose=True)
train(model, criterion, optimizer, scheduler, train_loader, val_loader, device, num_epochs=500)


'''
self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 1)
drop 0.2 2 layers
Training Loss: 0.0025, Validation Loss: 1.9067, Accuracy: 0.8013, Precision: 0.3000, Recall: 0.5000, F1-score: 0.3750, ROC AUC: 0.6711


self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 1)
Training Loss: 0.0005, Validation Loss: 1.3775, Accuracy: 0.8146, Precision: 0.3333, Recall: 0.5556, F1-score: 0.4167, ROC AUC: 0.7026
'''